In [1]:
# import some function we can use later
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import os
import sys
from IPython.display import display, Image
from six.moves import cPickle as pickle
%matplotlib inline


In [5]:
#load data first
pickle_file = '/home/htan/proj/TensorFlow/data/visibility/' +  'ASOS_alone.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset= save['train_dataset']
    validate_dataset = save['validate_dataset']
    test_dataset = save['test_dataset']
    test_old = save['test_dataset_evan']
    train_old = save['train_dataset_evan']
    del save
    

In [6]:
train_old_data = train_old['data']
train_old_label = train_old['label']
test_old_data = test_old['data']
test_old_label = test_old['label']

In [7]:
#dataset normalize
mean = train_old_data.mean(axis = 0)
std = train_old_data.std(axis = 0)
print(mean.shape, std.shape)
train_old_data_n = (train_old_data - mean)/std
test_old_data_n = (test_old_data - mean)/std

(70,) (70,)


In [8]:
train_old_label[:100]

array([  9.,  10.,  10.,   9.,   8.,   9.,   8.,   7.,   7.,   6.,   6.,
         7.,   7.,   9.,   8.,   7.,   7.,   7.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
         9.,  10.,  10.,  10.,   9.,  10.,   8.,   9.,   9.,   8.,   7.,
         7.,   8.,   9.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.])

In [10]:
from sklearn.preprocessing import Binarizer

In [11]:
pre = Binarizer(threshold = 1.01)
b_train_old_label = pre.transform(train_old_label.reshape(1, -1))
b_test_old_label = pre.transform(test_old_label.reshape(1, -1))

In [12]:
b_train_old_label[:, :30][0].shape

(30,)

In [13]:
c1 = b_train_old_label.sum(axis = 1)[0] 

In [14]:
c2 = b_train_old_label.shape[1] - c1

In [15]:
print(c1, c2, c2*100/(c1+c2))

199555.0 2715.0 1.34226528897


In [14]:
#need to balanced the dataset
from unbalanced_dataset.under_sampling import ClusterCentroids

In [15]:
c_train_old_label = 1 - b_train_old_label[0]
c_test_old_label = 1 - b_test_old_label[0]

In [18]:
CC = ClusterCentroids()
train_data_n_resample, c_train_label_resample = CC.fit_transform(train_old_data_n, c_train_old_label)

Determining classes statistics... 2 classes detected: Counter({0.0: 199555, 1.0: 2715})
Under-sampling performed: Counter({0.0: 2715, 1.0: 2715})


In [20]:
pickle_file = '/home/nfs/mjmu/haiming/data/visibility/' +  'CC_Under_sSample.pickle'

with open(pickle_file, 'wb') as f:
    save = {
        'train_data_n_resample': train_data_n_resample,
        'c_train_label_resample': c_train_label_resample
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
test_old_data_n_resample, c_test_old_label_resample = sm.fit_transform(test_old_data_n, c_test_old_label)

In [19]:
train_data_n_resample.shape, c_train_label_resample.shape

((5430, 70), (5430,))

In [21]:
c1 = c_train_label_resample.sum()
c2 = c_train_label_resample.shape[0] - c1
print(c1, c2, c1*100/(c1+c2))

2715.0 2715.0 50.0


In [16]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(train_data_n, c_train_label)

NameError: name 'train_data_n' is not defined

In [ ]:
predict_c = clf.predict(test_old_data_n)
TP = ((predict_c + c_test_old_label)>=2).astype(int).sum()
FP = predict_c.sum() - TP
FN = c_test_old_label.sum() - TP
print("TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % (TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))

In [23]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(train_data_n_resample, c_train_label_resample)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [28]:

predict_c = clf.predict(test_old_data_n_resample)
TP = ((predict_c + c_test_old_label_resample)>=2).astype(int).sum()
FP = predict_c.sum() - TP
FN = c_test_old_label_resample.sum() - TP
print("TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % (TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))

TP=844, FP=3743, FN=91, CSI=0.180, POD=0.903, FAR=0.816


In [29]:
predict_c = clf.predict(test_old_data_n)
TP = ((predict_c + c_test_old_label)>=2).astype(int).sum()
FP = predict_c.sum() - TP
FN = c_test_old_label.sum() - TP
print("TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % (TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))

TP=844, FP=3743, FN=91, CSI=0.180, POD=0.903, FAR=0.816


In [ ]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier()
clf.fit(train_data_n, c_train_label)

In [ ]:
predict_c = clf.predict(test_old_data_n)
print(predict_c.shape)
TP = ((predict_c + c_test_old_label)>=2).astype(int).sum()
FP = predict_c.sum() - TP
FN = c_test_old_label.sum() - TP
print("TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % ( TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))

In [30]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier()
clf.fit(train_data_n_resample, c_train_label_resample)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [31]:
predict_c = clf.predict(test_old_data_n_resample)
TP = ((predict_c + c_test_old_label_resample)>=2).astype(int).sum()
FP = predict_c.sum() - TP
FN = c_test_old_label_resample.sum() - TP
print("TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % (TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))

NameError: name 'test_old_data_n_resample' is not defined

In [32]:
predict_c = clf.predict(test_old_data_n)
print(predict_c.shape)
TP = ((predict_c + c_test_old_label)>=2).astype(int).sum()
FP = predict_c.sum() - TP
FN = c_test_old_label.sum() - TP
print("TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % ( TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))

(75852,)
TP=792, FP=5162, FN=143, CSI=0.130, POD=0.847, FAR=0.867


In [ ]:
from sklearn.tree import DecisionTreeClassifier
for max_d in range(2, 11):
    clf = DecisionTreeClassifier(max_depth = max_d)
    clf.fit(train_data_n, c_train_label)
    predict_c = clf.predict(test_old_data_n)
    TP = ((predict_c + c_test_old_label)>=2).astype(int).sum()
    FP = predict_c.sum() - TP
    FN = c_test_old_label.sum() - TP
    print("depth=%d, TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % (max_d, TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))

In [33]:
from sklearn.tree import DecisionTreeClassifier
for max_d in range(2, 11):
    clf = DecisionTreeClassifier(max_depth = max_d)
    clf.fit(train_data_n_resample, c_train_label_resample)
    predict_c = clf.predict(test_old_data_n)
    TP = ((predict_c + c_test_old_label)>=2).astype(int).sum()
    FP = predict_c.sum() - TP
    FN = c_test_old_label.sum() - TP
    print("depth=%d, TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % (max_d, TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))

depth=2, TP=877, FP=17268, FN=58, CSI=0.048, POD=0.938, FAR=0.952
depth=3, TP=900, FP=23415, FN=35, CSI=0.037, POD=0.963, FAR=0.963
depth=4, TP=900, FP=24034, FN=35, CSI=0.036, POD=0.963, FAR=0.964
depth=5, TP=900, FP=23263, FN=35, CSI=0.037, POD=0.963, FAR=0.963
depth=6, TP=900, FP=23532, FN=35, CSI=0.037, POD=0.963, FAR=0.963
depth=7, TP=903, FP=24335, FN=32, CSI=0.036, POD=0.966, FAR=0.964
depth=8, TP=908, FP=24560, FN=27, CSI=0.036, POD=0.971, FAR=0.964
depth=9, TP=907, FP=24445, FN=28, CSI=0.036, POD=0.970, FAR=0.964
depth=10, TP=911, FP=24646, FN=24, CSI=0.036, POD=0.974, FAR=0.964


In [ ]:
from sklearn.ensemble import RandomForestClassifier
for max_d in range(2, 21):
    clf = RandomForestClassifier(max_depth = max_d)
    clf.fit(train_data_n, c_train_label)
    predict_c = clf.predict(test_old_data_n)
    TP = ((predict_c + c_test_old_label)>=2).astype(int).sum()
    FP = predict_c.sum() - TP
    FN = c_test_old_label.sum() - TP
    print("depth=%d, TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % (max_d, TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='linear', C=1.0)
clf.fit(train_data_n, c_train_label)

In [ ]:
predict_c = clf.predict(test_old_data_n)
TP = ((predict_c + c_test_old_label)>=2).astype(int).sum()
FP = predict_c.sum() - TP
FN = c_test_old_label.sum() - TP
print(" TP=%d, FP=%d, FN=%d, CSI=%.3f, POD=%.3f, FAR=%.3f" % ( TP, FP, FN, TP/(TP + FP + FN), TP/(TP + FN), 1 - TP/(TP + FP)))